<a href="https://colab.research.google.com/github/NickStrauch13/beverage-recommender/blob/tom/beer_svd.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install -q kaggle

In [18]:
from google.colab import files

files.upload()
print("files done")

Saving kaggle.json to kaggle (1).json
files done


In [3]:
! mkdir ~/.kaggle

! cp kaggle.json ~/.kaggle/

In [4]:
! chmod 600 ~/.kaggle/kaggle.json

In [5]:
! kaggle datasets list

ref                                                         title                                           size  lastUpdated          downloadCount  voteCount  usabilityRating  
----------------------------------------------------------  ---------------------------------------------  -----  -------------------  -------------  ---------  ---------------  
sudarshan24byte/online-food-dataset                         Online Food Dataset                              3KB  2024-03-02 18:50:30          24254        484  0.9411765        
bhavikjikadara/student-study-performance                    Student Study Performance                        9KB  2024-03-07 06:14:09          12001        154  1.0              
sukhmandeepsinghbrar/housing-price-dataset                  Housing Price Dataset                          780KB  2024-04-04 19:45:43           1436         28  1.0              
muhammadkashif724/netflix-tv-shows-2021                     Netflix TV Shows 2021                        

In [6]:
!kaggle datasets download -d rdoume/beerreviews

100% 27.4M/27.4M [00:00<00:00, 43.9MB/s]
100% 27.4M/27.4M [00:00<00:00, 47.8MB/s]


In [8]:
!ls

beerreviews.zip  kaggle.json  sample_data


In [7]:
!unzip beerreviews.zip

Archive:  beerreviews.zip
  inflating: beer_reviews.csv        


In [2]:
import pandas as pd
df = pd.read_csv('../../data/beer_reviews.csv')

In [11]:
df.head()

,brewery_id,brewery_name,review_time,review_overall,review_aroma,review_appearance,review_profilename,beer_style,review_palate,review_taste,beer_name,beer_abv,beer_beerid
0,10325,Vecchio Birraio,1234817823,1.5,2.0,2.5,stcules,Hefeweizen,1.5,1.5,Sausa Weizen,5.0,47986
1,10325,Vecchio Birraio,1235915097,3.0,2.5,3.0,stcules,English Strong Ale,3.0,3.0,Red Moon,6.2,48213
2,10325,Vecchio Birraio,1235916604,3.0,2.5,3.0,stcules,Foreign / Export Stout,3.0,3.0,Black Horse Black Beer,6.5,48215
3,10325,Vecchio Birraio,1234725145,3.0,3.0,3.5,stcules,German Pilsener,2.5,3.0,Sausa Pils,5.0,47969
4,1075,Caldera Brewing Company,1293735206,4.0,4.5,4.0,johnmichaelsen,American Double / Imperial IPA,4.0,4.5,Cauldron DIPA,7.7,64883


In [3]:
df.beer_beerid.value_counts()

beer_beerid
2093     3290
412      3111
1904     3000
1093     2728
92       2704
         ... 
12708       1
70915       1
21441       1
62863       1
47986       1
Name: count, Length: 66055, dtype: int64

In [4]:
df.review_profilename.value_counts()

review_profilename
northyorksammy     5817
BuckeyeNation      4661
mikesgroove        4617
Thorpe429          3518
womencantsail      3497
                   ... 
fnong2                1
drewzline             1
amandacrane           1
mymasterpiece75       1
joeebbs               1
Name: count, Length: 33387, dtype: int64

In [5]:
df['review_overall'].value_counts()

review_overall
4.0    582764
4.5    324385
3.5    301817
3.0    165644
5.0     91320
2.5     58523
2.0     38225
1.5     12975
1.0     10954
0.0         7
Name: count, dtype: int64

In [6]:
ndf = df[['review_profilename', 'beer_beerid', 'review_overall']]

In [7]:
# Check for and handle missing values
print("Missing values before handling:")
print(ndf.isnull().sum())

# Optionally handle missing values, if any
ndf = ndf.dropna()  # This line removes any rows with missing data

print("Missing values after handling:")
print(ndf.isnull().sum())

# The dataframe is now ready for use in an SVD model
print(ndf.head())

Missing values before handling:
review_profilename    348
beer_beerid             0
review_overall          0
dtype: int64
Missing values after handling:
review_profilename    0
beer_beerid           0
review_overall        0
dtype: int64
  review_profilename  beer_beerid  review_overall
0            stcules        47986             1.5
1            stcules        48213             3.0
2            stcules        48215             3.0
3            stcules        47969             3.0
4     johnmichaelsen        64883             4.0


In [15]:
!pip install scikit-surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 kB 5.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp310-cp310-linux_x86_64.whl size=3162969 sha256=5958ad0df4abc0ebfe358bb9fbfe712bb3afc46ad7403461b4446c733d7a1822
  Stored in directory: /root/.cache/pip/wheels/a5/ca/a8/4e28def53797fdc4363ca4af740db15a9c2f1595ebc51fb445
Successfully built scikit-surprise


In [10]:
import pandas as pd
from surprise import SVD, Dataset, Reader
from surprise.model_selection import train_test_split, cross_validate
from surprise import accuracy

# Create a dictionary mapping beer IDs to names
beer_id_to_name = pd.Series(df.beer_name.values,index=df.beer_beerid).to_dict()

# Prepare the DataFrame for Surprise
reader = Reader(rating_scale=(0, 5))
data = Dataset.load_from_df(ndf[['review_profilename', 'beer_beerid', 'review_overall']], reader)

# Split the data into training and testing sets
trainset, testset = train_test_split(data, test_size=0.25, random_state=42)

# Train the SVD model
model = SVD()
model.fit(trainset)
predictions = model.test(testset)

# Calculate and print RMSE
accuracy.rmse(predictions)

# Cross-validation
results = cross_validate(model, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)
print(f"Cross validation: {results}")

RMSE: 0.6103
Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.6054  0.6063  0.6109  0.6090  0.6081  0.6079  0.0020  
MAE (testset)     0.4535  0.4537  0.4562  0.4556  0.4549  0.4548  0.0010  
Fit time          12.77   12.09   11.57   11.23   11.82   11.89   0.52    
Test time         1.52    1.43    1.41    1.30    1.33    1.40    0.08    
Cross validation: {'test_rmse': array([0.60537901, 0.6063035 , 0.61088927, 0.60899353, 0.60807135]), 'test_mae': array([0.45352628, 0.45370612, 0.45621524, 0.45556198, 0.4549185 ]), 'fit_time': (12.771265029907227, 12.085823059082031, 11.567872047424316, 11.230206966400146, 11.816775798797607), 'test_time': (1.515260934829712, 1.4327950477600098, 1.4052860736846924, 1.3047490119934082, 1.33101487159729)}


In [11]:
user_to_predict = 'stcules'
beers_to_predict = [47986, 64883]  # Example beer IDs to predict ratings for

for beer_id in beers_to_predict:
    prediction = model.predict(user_to_predict, beer_id)
    print(f"Predicted rating for {beer_id_to_name[beer_id]} (ID: {beer_id}): {prediction.est:.2f}")

Predicted rating for Sausa Weizen (ID: 47986): 2.83
Predicted rating for Cauldron DIPA (ID: 64883): 3.12


In [14]:
# Get the top N recommendations from a username using model.predict

from collections import defaultdict

def get_top_n_recommendations_with_predicted_rating(username, n=10):
    # First, get all beer IDs that the user has not rated
    rated_beer_ids = ndf[ndf.review_profilename == username].beer_beerid.unique()
    all_beer_ids = ndf.beer_beerid.unique()
    unrated_beer_ids = list(set(all_beer_ids) - set(rated_beer_ids))

    # Next, predict the rating for each of these beers
    predictions = [model.predict(username, beer_id) for beer_id in unrated_beer_ids]

    # Get the top N predictions
    top_predictions = sorted(predictions, key=lambda x: x.est, reverse=True)[:n]

    # Return the top N recommendations
    recommendations = defaultdict(list)
    for prediction in top_predictions:
        recommendations['beer_id'].append(prediction.iid)
        recommendations['beer_name'].append(beer_id_to_name[prediction.iid])
        recommendations['predicted_rating'].append(prediction.est)

    return pd.DataFrame(recommendations)

# Get the top 10 recommendations for the user 'stcules'
print(get_top_n_recommendations_with_predicted_rating('brentk56', n=10))







   beer_id                                       beer_name  predicted_rating
0    44910                                     Dirty Horse          4.827469
1    58610                                  Hoppy Birthday          4.817829
2    68548              Armand'4 Oude Geuze Lente (Spring)          4.806602
3    23413  Geuze Cuvée J&J (Joost En Jessie) Blauw (Blue)          4.777379
4    36316                         Cantillon Blåbær Lambik          4.736787
5    70356              Armand'4 Oude Geuze Zomer (Summer)          4.734385
6    47668       Imperial Eclipse Stout - Pappy Van Winkle          4.731553
7    44112                                      Yellow Bus          4.703693
8    56082                                      Citra DIPA          4.691855
9    63649                                     Rare D.O.S.          4.672207


In [25]:
# new function that returns the top n recommendations for a user but only for beers that have not been rated by the user and have at least a certain number of reviews

def get_top_n_recommendations_with_predicted_rating_and_min_reviews(username, n=10, min_reviews=10):
    # First, get all beer IDs that the user has not rated
    rated_beer_ids = ndf[ndf.review_profilename == username].beer_beerid.unique()
    all_beer_ids = ndf.beer_beerid.unique()
    unrated_beer_ids = list(set(all_beer_ids) - set(rated_beer_ids))

    # Next, predict the rating for each of these beers
    predictions = [model.predict(username, beer_id) for beer_id in unrated_beer_ids]

    # Get the top N predictions
    top_predictions = sorted(predictions, key=lambda x: x.est, reverse=True)

    # Return the top N recommendations
    recommendations = defaultdict(list)
    for prediction in top_predictions:
        # Check if the beer has at least min_reviews reviews
        if ndf[ndf.beer_beerid == prediction.iid].shape[0] >= min_reviews:
            recommendations['beer_id'].append(prediction.iid)
            recommendations['beer_name'].append(beer_id_to_name[prediction.iid])
            recommendations['predicted_rating'].append(prediction.est)
        if len(recommendations['beer_id']) == n:
            break

    return pd.DataFrame(recommendations)

# Get the top 10 recommendations for the user 'stcules' with a minimum of 10 reviews per beer
print(get_top_n_recommendations_with_predicted_rating_and_min_reviews('brentk56', n=10, min_reviews=100))


   beer_id                                beer_name  predicted_rating
0    36316                  Cantillon Blåbær Lambik          4.736787
1    56082                               Citra DIPA          4.691855
2    41928  Deviation - Bottleworks 9th Anniversary          4.645943
3    51116                          Duck Duck Gooze          4.615239
4    16814                             Heady Topper          4.602751
5    42349              Vanilla Bean Aged Dark Lord          4.586296
6    44942                 Unplugged Bohemian Lager          4.583295
7    62172                                  Ephraim          4.568585
8    64228                              Zombie Dust          4.567147
9    34313  Kuhnhenn Bourbon Barrel Fourth Dementia          4.528902


In [27]:
# save the model
import pickle
filename = '../../models/finalized_model.sav'
pickle.dump(model, open(filename, 'wb'))

# load the model
loaded_model = pickle.load(open(filename, 'rb'))

# Get the top 10 recommendations for the user 'stcules' with a minimum of 10 reviews per beer

def get_top_n_recommendations_with_predicted_rating_and_min_reviews_and_model(username,model,ndf, n=10, min_reviews=10, ):
    # First, get all beer IDs that the user has not rated
    rated_beer_ids = ndf[ndf.review_profilename == username].beer_beerid.unique()
    all_beer_ids = ndf.beer_beerid.unique()
    unrated_beer_ids = list(set(all_beer_ids) - set(rated_beer_ids))

    # Next, predict the rating for each of these beers
    predictions = [model.predict(username, beer_id) for beer_id in unrated_beer_ids]

    # Get the top N predictions
    top_predictions = sorted(predictions, key=lambda x: x.est, reverse=True)

    # Return the top N recommendations
    recommendations = defaultdict(list)
    for prediction in top_predictions:
        # Check if the beer has at least min_reviews reviews
        if ndf[ndf.beer_beerid == prediction.iid].shape[0] >= min_reviews:
            recommendations['beer_id'].append(prediction.iid)
            recommendations['beer_name'].append(beer_id_to_name[prediction.iid])
            recommendations['predicted_rating'].append(prediction.est)
        if len(recommendations['beer_id']) == n:
            break

    return pd.DataFrame(recommendations)

# Get the top 10 recommendations for the user 'stcules' with a minimum of 10 reviews per beer
print(get_top_n_recommendations_with_predicted_rating_and_min_reviews_and_model('brentk56', n=10, min_reviews=100, model=loaded_model, ndf=ndf))




   beer_id                                beer_name  predicted_rating
0    36316                  Cantillon Blåbær Lambik          4.736787
1    56082                               Citra DIPA          4.691855
2    41928  Deviation - Bottleworks 9th Anniversary          4.645943
3    51116                          Duck Duck Gooze          4.615239
4    16814                             Heady Topper          4.602751
5    42349              Vanilla Bean Aged Dark Lord          4.586296
6    44942                 Unplugged Bohemian Lager          4.583295
7    62172                                  Ephraim          4.568585
8    64228                              Zombie Dust          4.567147
9    34313  Kuhnhenn Bourbon Barrel Fourth Dementia          4.528902
